Basemodel: DistilBERT

In [ ]:
!git clone https://github.com/drryodino246/csc542-gp-team164.git

Cloning into 'csc542-gp-team164'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 46 (delta 26), reused 11 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (46/46), 676.32 KiB | 2.07 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
from datasets import load_dataset
emotions = load_dataset('csv', data_files='/content/csc542-gp-team164/baseline.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
emotions_split = emotions['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = emotions_split['train']
test_dataset = emotions_split['test']

print(f"Training data size: {len(train_dataset)}")
print(f"Test data size: {len(test_dataset)}")

Training data size: 15347
Test data size: 3837


In [ ]:
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True, max_length=512, return_tensors="pt")

# Tokenize both the train and test datasets
train_encoded = train_dataset.map(tokenize, batched=True)
test_encoded = test_dataset.map(tokenize, batched=True)

# Rename 'updated_emotion' to 'label' for both train and test datasets
train_encoded = train_encoded.rename_column("updated_emotion", "label")
test_encoded = test_encoded.rename_column("updated_emotion", "label")

# Check the first few samples from the tokenized train and test datasets
print("First few samples from train_encoded:")
print(train_encoded[0])  # Inspect the first entry in the train dataset

print("\nFirst few samples from test_encoded:")
print(test_encoded[0])  # Inspect the first entry in the test dataset

Map:   0%|          | 0/3837 [00:00<?, ? examples/s]

First few samples from train_encoded:
{'text': 'I was so mad when I got detention for something I did not do. The kid who did it had nothing bad happen to him either', 'emotion': 'furious', 'label': 'very negative', 'input_ids': [101, 1045, 2001, 2061, 5506, 2043, 1045, 2288, 12345, 2005, 2242, 1045, 2106, 2025, 2079, 1012, 1996, 4845, 2040, 2106, 2009, 2018, 2498, 2919, 4148, 2000, 2032, 2593, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

First few samples from test_encoded:
{'text': 'My former roomate left her cat keychain here. We fought a lot, but I kinda miss her now.', 'emotion': 'sentimental', 

In [ ]:
# Map string labels to integer labels
def map_labels(example):
    label_map = {
        "very negative": 0,
        "negative": 1,
        "positive": 2,
        "very positive": 3
    }
    example["label"] = label_map[example["label"]]  # Replace string label with integer label
    return example

# Apply the label mapping function to both train and test datasets
train_encoded = train_encoded.map(map_labels)
test_encoded = test_encoded.map(map_labels)

# Now the 'label' column will contain integers instead of strings
print("First few samples from train_encoded (with mapped labels):")
print(train_encoded[:2])  # Check the first 2 samples after label mapping

Map:   0%|          | 0/15347 [00:00<?, ? examples/s]

Map:   0%|          | 0/3837 [00:00<?, ? examples/s]

First few samples from train_encoded (with mapped labels):
{'text': ['I was so mad when I got detention for something I did not do. The kid who did it had nothing bad happen to him either', 'Today has been a hard day! Partner is away at therapy for?? Goodness knows when..and he was a bit cranky yesterday, so that makes me feel even more alone.'], 'emotion': ['furious', 'lonely'], 'label': [0, 1], 'input_ids': [[101, 1045, 2001, 2061, 5506, 2043, 1045, 2288, 12345, 2005, 2242, 1045, 2106, 2025, 2079, 1012, 1996, 4845, 2040, 2106, 2009, 2018, 2498, 2919, 4148, 2000, 2032, 2593, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2651, 2038, 2042, 1037, 2524, 2154, 999, 4256, 2003, 2185, 2012, 7242, 2005, 1029, 1029, 15003, 4282, 2043, 1012, 1012, 1998, 2002, 2001, 1037, 2978, 27987, 2100, 7483, 1010, 2061, 2008, 3084, 2033, 2514, 2130, 2062, 2894, 1012, 102, 0, 0, 0, 

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 4
id2label = {
    "0": "very negative",
    "1": "negative",
    "2": "positive",
    "3": "very positive"
}
label2id = {
    "very negative": 0,
    "negative": 1,
    "positive": 2,
    "very positive": 3
}

model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels, id2label=id2label, label2id=label2id).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install tensorboard

In [ ]:
from transformers import TrainingArguments
import tensorflow as tf
from torch.utils.tensorboard import SummaryWriter

batch_size = 64
logging_steps = 500
model_name = f"{model_ckpt}-finetuned-emotion"

training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir=f"{model_name}/logs",  # Directory for logs
    logging_steps=logging_steps,  # Log training progress every 'logging_steps'
    load_best_model_at_end=True,  # Load the best model based on evaluation metrics
    metric_for_best_model="f1",  # Track the F1 score for selecting the best model
    greater_is_better=True,  # We want higher F1 score, so set this to True
    disable_tqdm=False,  # Keep TQDM for progress bar
    report_to="tensorboard",  # Enable TensorBoard logging
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(p):
    preds, labels = p
    preds = np.argmax(preds, axis=1)

    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_encoded,
    eval_dataset=test_encoded,
    tokenizer=tokenizer
)

<ipython-input-25-8a39fc65cbe4>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.788064,0.669273,0.665298,0.668193,0.669273
2,No log,0.745937,0.691426,0.690266,0.690267,0.691426


TrainOutput(global_step=480, training_loss=0.8382356643676758, metrics={'train_runtime': 332.1193, 'train_samples_per_second': 92.419, 'train_steps_per_second': 1.445, 'total_flos': 1000045513686240.0, 'train_loss': 0.8382356643676758, 'epoch': 2.0})

In [ ]:
trainer.save_model()

In [ ]:
from tensorboard import notebook
notebook.display("logs")

TypeError: %d format: a real number is required, not str

References:
- Fine-Tuning DistilBERT for Emotion Classification, (for coding assistance)
https://medium.com/@ahmettsdmr1312/fine-tuning-distilbert-for-emotion-classification-84a4e038e90e
- ChatGPT (for coding assistance)
